In [1]:
import tkinter as tk
from tkinter import messagebox
import socket
import random


RSA CODE

In [8]:
def is_prime(num):
    if num == 1:
        return 0
    elif num == 2:
        return 1
    elif num > 2 and num % 2 == 0:
        return 0
    else:
        for i in range(3, int(num**0.5)+1, 2):
            if num % i == 0:
                return 0
        return 1

def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)
def getd(e, on):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = on
    
    while e > 0:
        temp1 = temp_phi // e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2 - temp1 * x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y
    
    if temp_phi == 1:
        return d

def RSA_ENC(text):
        while True:
            p = random.randrange(1,50000)
            q = random.randrange(1,50000)
            if   is_prime(p)==1 and is_prime(q)==1:
                print(p)
                print(q)
                break
        n= p*q
        on= (p-1) *  (q-1)
        e = random.randrange(1, on)
        g = gcd(e, on)
        while g != 1:
            e = random.randrange(1, on)
            g = gcd(e, on)
        print(e)
        d= getd(e,on)
        cipher = [pow(ord(t), e, n) for t in text]
        # enc=''
        # for c in text:
        #     enc += pow(ord(c),e)%n
        print(''.join(map(lambda x: str(x), cipher)))
        # x = cipher + ','+ str(d)+','+ str(n)
        # x = cipher + ',' + str(d) + ',' + str(n)
        cipher_str = ','.join(map(str, cipher))
        d_str = str(d)
        n_str = str(n)
        concatenated = cipher_str + '|' + d_str + '|' + n_str
        return concatenated
        
    
    
def RSA_Dec(concatenated):
    cipher_str, d_str, n_str = concatenated.split('|')
    cipher = [int(c) for c in cipher_str.split(',')]
    d = int(d_str)
    n = int(n_str)
    plain = [chr(pow(c, d, n)) for c in cipher]
    return ''.join(plain)

DEFIHILMAN kEY GENERATION

In [9]:
def client1():
    pk=80
    P,G= P_G()
    PUBK=calc_PUBK(pk,G,P)
    c = str(P)+'|'+str(G)+'|'+str(PUBK)
    return c,P,G

def  client_DEC(PUBK,P,G):
    pk=90
    PUBK2=calc_PUBK(pk,int(G),int(P))
    session=calc_session_key(int(PUBK),pk,int(P))
    return session

SEND BUTTON 

In [10]:

def button_click():
    global button_clicked
    if not button_clicked:
         host = '192.168.43.51'
         port = 5050
         TEXT="011key"
         global p_h
         global g_h
         c,p_h,g_h  =  client1()
         button_clicked = True
         client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
         client_socket.connect((host, port))
         client_socket.send(TEXT.encode())
         client_socket.send(c.encode())
         client_socket.close()
    else:
        text = textbox.get()
        if text !="":
            selected_value = dropdown_var.get()
            if selected_value=='RSA':
                    c=RSA_ENC(text)
                    TEXT="011rsa"
            elif selected_value=='Elgamal':
                c=GAMAL_encrypt(text)
                TEXT="011elgamal"
            elif selected_value=='DES':
                c=DES_ENCRYPT2(text)
                TEXT="011des"
            elif selected_value=='AES':
                TEXT="011aes"
                c=EncrAES(text)

            display_text = display_label.cget("text")
            display_label.config(text=display_text + "\n"+"\n" + text)
            textbox.delete(0, tk.END)
            host = '192.168.43.51'
            port = 5050

            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((host, port))




            client_socket.send(TEXT.encode())

            client_socket.send(c.encode())


            client_socket.close()
        else:
            print("empty") 

GUI 

In [11]:


import tkinter as tk
import time
pk=80
p_h=0
g_h=0
sessionkey= 0 
pubk=0
pubk2=0
button_clicked = False
def show_time():
    host = '192.168.43.51'
    port = 5050
    message="11"
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    client_socket.send(message.encode())
    data = client_socket.recv(4096).decode()
    print(recieved_text.cget("text"))
    if data in recieved_text.cget("text"):
        pass
    else:
        if data[:3]=='key':
            pubk2= data[3:]
        elif data[:3] == 'rsa':
            data =  data[3:]
            d = RSA_Dec(data)
            if d in recieved_text.cget("text"):
                pass
            else:
                print("RSA IS::", d)
                data=  d
                display_text = recieved_text.cget("text")
                recieved_text.config(text=display_text + "\n"+"\n" + data)
        elif data[:7] == 'elgamal': 
            data =  data[7:]
            d = GAMAL_decrypt(data)
            if d in recieved_text.cget("text"):
                pass
            else:
                print("gamal  IS::", d)
                data=  d
                display_text = recieved_text.cget("text")
                recieved_text.config(text=display_text + "\n"+"\n" + data)
        elif data[:3] == 'des': 
            data =  data[3:]
            d = DES_DECRYPT(data)
            if d in recieved_text.cget("text"):
                pass
            else:
                print("DES IS::", d)
                data=  d
                display_text = recieved_text.cget("text")
                recieved_text.config(text=display_text + "\n"+"\n" + data)
        elif data[:3] == 'aes': 
            data =  data[3:]
            d = DecryAES(data)
            if d in recieved_text.cget("text"):
                pass
            else:
                print("aes IS::", d)
                data=  d
                display_text = recieved_text.cget("text")
                recieved_text.config(text=display_text + "\n"+"\n" + data)
        # else:
        #     print("Received from server:", data)
        #     display_text = recieved_text.cget("text")
        #     recieved_text.config(text=display_text + "\n"+"\n" + data)
    client_socket.close()

    recieved_text.after(1000, show_time)


root = tk.Tk()

root.title("Security Project_C1")
root.geometry("500x700")
canvas = tk.Canvas(root, width=500, height=700)
canvas.pack()
background_image = tk.PhotoImage(file="b.png")
canvas.create_image(0, 0, anchor=tk.NW, image=background_image)
button_clicked = False
label = tk.Label(root, text="Enter text:")
label.place(x=0, y=640)
textbox = tk.Entry(root)
textbox = tk.Entry(root, width=50)
textbox.place(x=0,y=670)

recieved_text = tk.Label(root, text="\n", fg="blue")
recieved_text.place(x=300, y=10)     
display_label = tk.Label(root , text="")
display_label.place(x=220, y=10)
show_time()
button = tk.Button(root, text="Send", command=button_click)
# button.pack(side=tk.BOTTOM)
button.place(x=320,y=670)
##################################################
# button2 = tk.Button(root, text="Refresh", command=button2_click)

# button2.place(x=10,y=30)
#####################################################
options = ["AES", "DES", "RSA","Elgamal"]
dropdown_var = tk.StringVar(root)
dropdown_var.set(options[0])
dropdown = tk.OptionMenu(root, dropdown_var, *options)
dropdown.place(x=400,y=666)
# refresh_w

root.mainloop()











































Number Is Not Prime, Please Enter Again!
Number Is Not Prime, Please Enter Again!
Number Is Not Prime, Please Enter Again!
Number Is Not Prime, Please Enter Again!
290
Number Is Not A Primitive Root Of 631, Please Try Again!
539
Number Is Not A Primitive Root Of 631, Please Try Again!
595
Number Is Not A Primitive Root Of 631, Please Try Again!
409














The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

6

011001100110000101110010011011110111010101101011
Encryption:   Round 1
10000010111000001011101000101000 

Encryption:   Round 2
10001011111111001011011001001011 

Encryption:   Round 3
11111011111111110010001111100010 

Encryption:   Round 4
10100001010101011010111101111010 

Encryption:   Round 5
01100101110011101100110011100110 

Encryption:   Round 6
11001101100001111011000010111100 

Encryption:   Round 7
11000001000100001000010111101001 

Encry




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111

...




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111




abood
The key after the PC-1 

00000000000000000000000000000000000000000000000000000000 

The 16 sub keys for the encryption operartion are: 

Decryption:   Round 1
10110111101101001000110111100011 

Decryption:   Round 2
10111001010011100000111010001110 

Decryption:   Round 3
10111110000010011011001101100100 

Decryption:   Round 4
01010111100110000011100110000000 

Decryption:   Round 5
01100011111011000010110000001011 

Decryption:   Round 6
10001101111010100111101101111101 

Decryption:   Round 7
01010001110101001110000001101101 

Decryption:   Round 8
10011110101001101111100010110001 

Decryption:   Round 9
00000011011010011100111111111111 

Decryption:   Round 10
01000111111111100110100100100000 

Decryption:   Round 11
01111000011001001110000010000110 

Decryption:   Round 12
01011000100110010011010010110110 

Decryption:   Round 13
10000011011111101000011100100110 

Decryption:   Round 14
00001111110100100011101010000001 

Decryption:   Round 15
0000000011111000011000000111